In [2]:
## imports
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (
    StratifiedKFold,  # use X = np.zeros(n_samples) in .split
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

from data_img import Img_Obj_Dataset

# if you have indeces and want knns from large set, can make an idx array and NearestVectorCaller._call_vec_set; can just train/test split np.arange(len(ds)) to get back indeces for large sets


from btv import *

In [3]:
## Data setup

ds = Img_Obj_Dataset("data/hardhat/test", max_dim=256)
# can pass metric == "cosine", get from NearestVectorCaller.metric on large sets
knn = KNeighborsClassifier(weights="distance", n_neighbors=10) #, metric = "cosine")
# X, y = ds.whole_img_vec_set[:] # only need y for class balancing... ds.whole_img_vec_set.y

idx_train, idx_test = train_test_split(
    np.arange(len(ds.whole_img_vec_set)), test_size=0.2
)

Detemining all classes...: 100%|██████████████████████████████| 1766/1766 [00:01<00:00, 1522.48it/s]


In [4]:
knn.fit(ds.whole_img_vec_set.X[idx_train], ds.whole_img_vec_set.y[idx_train])
# knn.predict(X_test)
# ds.whole_img_vec_set.X[idx_test]

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [5]:
accuracy_score(
    ds.whole_img_vec_set.y[idx_test], knn.predict(ds.whole_img_vec_set.X[idx_test])
)

0.748587570621469

In [9]:
# only include p if y==1 or, reeally, want p(right answer) so take elemnt of predicted prob indicated by binary y.
def prediction_entropy(y_true, y_predicted): # multilable
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    A = np.zeros_like(y_true)
    with np.errstate(divide="ignore"):
        for i, (pred, true) in enumerate(zip(y_predicted, y_true)):
            for j, (p, t) in enumerate(zip(pred, true)):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                A[i, j] = I
    return np.mean(A, axis=0)


# probably actually want to do this at the prediction level since that is the big/slow bit.
def prediction_entropy_lg(y_true, y_predicted): # sinlge label
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    with np.errstate(divide="ignore"):
        for pred, true in zip(y_predicted, y_true):
            for p, t in zip(pred, true):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                yield I

In [40]:
A = knn.predict(ds.whole_img_vec_set.X[idx_test])
P = knn.predict_proba(
    ds.whole_img_vec_set.X[idx_test]
)  # predicts the probability of each class. For whole image hardhats it is a list of 3 (p(no), P(yes)) arrays

In [41]:
prediction_entropy(y_predicted, y_true)

AttributeError: 'list' object has no attribute 'shape'

In [17]:
kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
# can just add all the labels to make it multiclass for splitting
entropies = []
running_train = []
for k, (train_idx, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
    # knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    # y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[test_idx])
    # y_true = ds.whole_img_vec_set.y[test_idx]
    # H = prediction_entropy(y_true, y_predicted)
    # entropies.append(H)
    # print(f"Fold : {k+1}, " f"Fold entropy : {H}")

In [20]:
np.mean(entropies, axis=0), np.std(entropies, axis=0), np.std(
    entropies, axis=0
) / np.mean(entropies, axis=0)

(array([75.04532578, 57.61076487, 16.80113314]),
 array([38.20580196, 28.99206553,  8.40056667]),
 array([0.50910302, 0.50324042, 0.50000001]))

In [188]:
knn.predict(ds.whole_img_vec_set.X[test_idx])

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [100]:
np.sum(ds.whole_img_vec_set.y, axis=0)

array([ 339, 1604,   45])

In [43]:
## with straigh vectorization of whole images, the knn accuracy goes down as more data is added. Therefore that vecotrization is shit. Maybe we try tabular data?

kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
train_idx = np.array([], int)
running_train = []
for k, (_, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
running_test = running_train.pop()
for k, old_test_idx in enumerate(running_train):
    train_idx = np.append(train_idx,old_test_idx)
    knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[running_test])
    y_true = ds.whole_img_vec_set.y[running_test]
    H = prediction_entropy(y_true, y_predicted)
    entropies.append(H)
    print(f"Fold : {k+1}, " f"Test set entropy : {np.mean(H)}", f"Train samples : {len(train_idx)}", f"Accuracy : {accuracy_score(y_true,knn.predict(ds.whole_img_vec_set.X[test_idx]))}")

Fold : 1, Test set entropy : 45.036827195467424 Train samples : 354 Accuracy : 0.7705382436260623
Fold : 2, Test set entropy : 45.040604343720496 Train samples : 707 Accuracy : 0.7648725212464589
Fold : 3, Test set entropy : 47.038715769593956 Train samples : 1060 Accuracy : 0.7620396600566572
Fold : 4, Test set entropy : 52.03305004721435 Train samples : 1413 Accuracy : 0.7592067988668555


In [42]:
hs, ss = add_stratified_folds_test(ds.whole_img_vec_set.X,ds.whole_img_vec_set.y,knn)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [294]:
imc = ds.imc[:100]
color = len(imc[0].shape) > 2
channels = imc[0].shape[2] if color else 1
    
px_mean, px_std = np.zeros(channels), np.zeros(channels)
running_sum,running_sumsq  = 0,0 
for n, im in tqdm(
    enumerate(imc), desc="Scanning to find mean and STD of pixel values...", unit="Image file"
):

    last_px_mean = px_mean.copy()
    last_px_std = px_std.copy()
    px_mean += (im.mean(0).mean(0) - px_mean)/(n + 1)
    px_std = last_px_std**2 + ((im - last_px_mean)*(im - px_mean)) if n>0 else im.std(0).std(0)

    print(im.std(0).std(0), np.power(px_std,0.25))

Scanning to find mean and STD of pixel values...: 5Image file [00:00, 38.50Image file/s]

[ 7.90666386  9.60787959 13.04808314] [1.67686584 1.76058456 1.90058229]
[13.55709516 14.75199134 11.17818635] [2.95329493 3.21669733 3.67009808]
[6.90468828 5.41286144 7.95601727] [ 8.72455141 10.34836303 13.47043371]
[14.77463388 15.5822074  15.94772289] [ 76.11780571 107.08862051 181.4525851 ]
[6.21629902 7.06672287 6.85317009] [ 5793.92034626 11467.97264307 32925.04063978]
[12.64741072 11.88559793 10.67692722] [3.35695130e+07 1.31514397e+08 1.08405830e+09]
[ 9.13088369  7.79770479 10.15771115] [1.12691220e+15 1.72960365e+16 1.17518240e+18]
[19.49760117 21.30235558 24.20416498] [1.26993111e+30 2.99152879e+32 1.38105367e+36]
[7.21893283 5.31042615 4.08973723] [1.61272502e+60 8.94924447e+64 1.90730925e+72]


/tmp/ipykernel_1652832/2649057096.py:14: RuntimeWarning: overflow encountered in square
  px_std = last_px_std**2 + np.sqrt((im - last_px_mean).std(0).std(0)*(im - px_mean).std(0).std(0)) if n>0 else im.std(0).std(0)
Scanning to find mean and STD of pixel values...: 20Image file [00:00, 60.93Image file/s]

[5.99678907 5.96440268 5.4661658 ] [inf inf inf]
[11.93698443 14.38026169 12.4476946 ] [inf inf inf]
[11.1866982  15.65115971 19.47195764] [inf inf inf]
[ 8.3269579   9.09830793 11.85552975] [inf inf inf]
[6.26337131 7.18232707 8.14203041] [inf inf inf]
[12.5590151  11.01058936 10.56337429] [inf inf inf]
[19.43823123 15.0431283  13.10619648] [inf inf inf]
[18.00949474 14.34814403 14.36994434] [inf inf inf]
[20.87913179 19.3901134  18.2688246 ] [inf inf inf]
[20.74493443 18.66062101 17.39873607] [inf inf inf]
[14.88889945 12.16057277 19.11420481] [inf inf inf]
[14.20288912 15.49543793 15.65908415] [inf inf inf]
[16.2779994  17.37303075 16.86397622] [inf inf inf]
[8.47176505 7.37632828 4.2184585 ] [inf inf inf]
[12.40394422 12.36313081 12.26807462] [inf inf inf]
[16.04419313 11.67137399 11.38104525] [inf inf inf]


Scanning to find mean and STD of pixel values...: 37Image file [00:00, 72.99Image file/s]

...

[8.34533229 6.86560778 5.51305149] [inf inf inf]
[ 6.20473239 12.99220019 12.81456903] [inf inf inf]
[19.52725301 13.13795618 16.4151293 ] [inf inf inf]
[14.29490146 14.68841004 16.81358742] [inf inf inf]
[ 9.48359954 10.94650384 11.42165178] [inf inf inf]
[10.39444146  9.35960764  9.25650439] [inf inf inf]
[7.45103412 5.16888229 4.19370589] [inf inf inf]
[12.57996505 10.56057292 13.62195658] [inf inf inf]
[6.93788468 6.96598312 8.66534688] [inf inf inf]
[13.28660178 12.92075147 11.86545145] [inf inf inf]
[11.1473738  10.88099313 11.22183291] [inf inf inf]
[19.78440086 15.75998719 16.56602446] [inf inf inf]
[11.13117351 10.35510793 11.27331073] [inf inf inf]
[12.65453917 13.97935161 19.58626799] [inf inf inf]
[19.77790396 19.47407989 17.56692634] [inf inf inf]
[ 8.08016713  5.758107   12.71294798] [inf inf inf]


Scanning to find mean and STD of pixel values...: 85Image file [00:01, 76.18Image file/s]

[13.23665395 13.14895635 10.85508992] [inf inf inf]
[19.75381961 18.52966888 16.89569107] [inf inf inf]
[7.50968573 6.53607978 6.8666064 ] [inf inf inf]
[20.59364975 13.54856246 12.37763712] [inf inf inf]
[14.66746034 12.32997023 11.34680107] [inf inf inf]
[12.15987482 11.98476475 13.7371797 ] [inf inf inf]
[11.35600306 10.19731277  9.15041907] [inf inf inf]
[11.0562424  10.23920744 12.0607973 ] [inf inf inf]
[16.02792827 14.48504998 15.65523333] [inf inf inf]
[13.55405625  8.05863814  8.93940725] [inf inf inf]
[15.12307124 17.03720127 13.28645563] [inf inf inf]
[7.79208036 8.02357514 8.11411706] [inf inf inf]
[12.13386615 12.20543905 12.53513635] [inf inf inf]
[4.98325927 4.49083084 4.51570334] [inf inf inf]
[16.00040242 14.9442788  14.35255822] [inf inf inf]
[14.16409217 15.81907667 15.11124991] [inf inf inf]


Scanning to find mean and STD of pixel values...: 100Image file [00:01, 71.99Image file/s]

[11.92493446 11.06964217 10.44536703] [inf inf inf]
[11.9077242  11.16095927 11.54814551] [inf inf inf]
[9.74178358 8.85670157 8.79207767] [inf inf inf]
[13.2267355 15.0146102 19.0529009] [inf inf inf]
[ 9.17911084  9.30680319 10.34834458] [inf inf inf]
[17.30090793 17.09700772 15.85485898] [inf inf inf]
[13.58227365 13.30490052 13.37131611] [inf inf inf]
[5.86135708 6.21799684 5.51019783] [inf inf inf]
[ 7.2252405   9.22855571 10.21652241] [inf inf inf]
[21.13545819 21.74986241 22.56041571] [inf inf inf]
[15.04612477 10.80686228 12.48166958] [inf inf inf]
